# **Weather Prediction ML model**

## Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, classification_report
import joblib

### Load Dataset

In [4]:
from google.colab import files
uploaded = files.upload()


Saving archive (1).zip to archive (1) (1).zip


In [5]:
import zipfile
import os

# Get the name of the uploaded zip file
zip_file_name = list(uploaded.keys())[0]

# Extract the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall()

csv_file_path = 'seattle-weather.csv'

# Check if the CSV file exists after extraction
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    display(df.head())
else:
    print(f"Error: '{csv_file_path}' not found after extraction.")

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [6]:
df.isnull().sum()    #Checking missing values

,0
date,0
precipitation,0
temp_max,0
temp_min,0
wind,0
weather,0


### Feature Enginnering

In [ ]:
df['humidity'] = 100 - (df['temp_max'] - df['temp_min']) * 2     # Derive humidity (approx formula)
df['humidity'] = df['humidity'].clip(0, 100)

In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])
df['day_of_year'] = df['date'].dt.dayofyear

In [ ]:
# Select Features and Targets
features = ['temp_max', 'temp_min', 'precipitation', 'wind', 'humidity', 'day_of_year']
target_numeric = ['temp_max', 'temp_min', 'humidity', 'wind']
target_weather = 'weather'

In [ ]:
# Split Data
X = df[features]
y_reg = df[target_numeric]           # For Linear Regression
y_cls = df[target_weather]           # For Logistic Regression